In [20]:
import numpy as np
import os
import pandas as pd

folder_path = "/mnt/B-SSD/maltamed/SAM-zero-shot/code/all_dice"

for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        #check if there is already a json file
        if os.path.isfile(f'{filename[:-4]}.json'):
            continue
        # Read the CSV file
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        
        #remove the image column
        df = df.drop(columns=['image'])
        #remove the first column
        df = df.drop(df.columns[0], axis=1)
        #remove brackets from the whole df
        df = df.replace('\[','',regex=True)
        df = df.replace('\]','',regex=True)
        
        #convert all values to float
        df = df.astype(float)
        
        #for each column, calculate the mean

        # Compute the mean of the highest values in each group and add it to the bottom row
        group_names = ["dice_rc", "dice_rs3", "dice_rs5", "dice_cp", "dice_bb", "dice_bbs_05", "dice_bbs_1", "dice_bbs_2"]
        highest_means = []
        

        for group_name in group_names:
            columns = [f"{group_name}_{i}" for i in range(3)]
            
            df[f'{group_name}_max'] = df.apply(lambda x: x[columns].max(), axis=1)
            
        # get the mean of all columns in a json
        means = df.mean().to_json(f'{filename[:-4]}.json')
         
        # Save the modified DataFrame to a new CSV file with prefix "modified_"
        # modified_filename = f"modified_{filename}"
        # modified_file_path = os.path.join(folder_path, modified_filename)
        # df.to_csv(modified_file_path, index=False)

In [25]:
#GETTING COMPLETE TABLE

import os
import json
import pandas as pd

# Read JSON files from folder
folder_path = '/mnt/B-SSD/maltamed/SAM-zero-shot/code/all_dice'
json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]

data = []

for file in json_files:
    with open(os.path.join(folder_path, file), 'r') as f:
        json_data = json.load(f)

    # Parse file name to extract dataset and model
    dataset, model = file.replace(".json", "").split("_values", 1)[1].split("_", 1)
    
    # Extract data from JSON
    record = {"dataset": dataset, "model": model}
    for key, value in json_data.items():
        if "max" not in key:
            prefix, pred = key.split("_", 1)
            if prefix.startswith('dice'):
                record.update({f"{prefix}_{pred}": value})
    data.append(record)

# Create DataFrame
df = pd.DataFrame(data)

# Pivot DataFrame to create the desired table format
table = df.melt(id_vars=["dataset", "model"], var_name="metric_prediction", value_name="value")
table[['metric', 'prediction']] = table['metric_prediction'].str.rsplit("_", n=1, expand=True)
table = table.pivot_table(index=["dataset", "model", "prediction"], columns="metric", values="value").reset_index()
table.columns.name = None

# Reorder columns and remove "dice_" from metric names
columns = ['dataset', 'model', 'prediction', 'dice_cp', 'dice_rc', 'dice_rs3', 'dice_rs5', 'dice_bb', 'dice_bbs_05', 'dice_bbs_1', 'dice_bbs_2']
table = table[columns]
table.columns = [col.replace('dice_', '') for col in table.columns]

# Sort the DataFrame based on model and dataset
table['model'] = pd.Categorical(table['model'], categories=['vit-h', 'vit-l', 'vit-b'], ordered=True)
table.sort_values(['dataset', 'model', 'prediction'], inplace=True)
table.reset_index(drop=True, inplace=True)

print(table)

#save table
table.to_csv('/mnt/B-SSD/maltamed/SAM-zero-shot/code/all_dice.csv', index=False)


   dataset  model prediction        cp        rc       rs3       rs5   
0      CVC  vit-h          0  0.715650  0.762991  0.860588  0.880438  \
1      CVC  vit-h          1  0.553970  0.543816  0.641503  0.754285   
2      CVC  vit-h          2  0.232323  0.223558  0.223715  0.245237   
3      CVC  vit-l          0  0.497890  0.482082  0.507819  0.522131   
4      CVC  vit-l          1  0.702237  0.727557  0.836015  0.841383   
..     ...    ...        ...       ...       ...       ...       ...   
58  mamoUS  vit-l          1  0.566997  0.535854  0.747646  0.779414   
59  mamoUS  vit-l          2  0.227964  0.205172  0.201842  0.252489   
60  mamoUS  vit-b          0  0.202206  0.191985  0.181030  0.212782   
61  mamoUS  vit-b          1  0.633775  0.603890  0.681784  0.691153   
62  mamoUS  vit-b          2  0.561960  0.521942  0.772828  0.796990   

          bb    bbs_05     bbs_1     bbs_2  
0   0.889461  0.880619  0.834852  0.702235  
1   0.926124  0.923735  0.915903  0.844092  


In [26]:
#GETTING COMPLETE TABLE MAX

import os
import json
import pandas as pd

# Read JSON files from folder
folder_path = '/mnt/B-SSD/maltamed/SAM-zero-shot/code/all_dice'
json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]

data = []

for file in json_files:
    with open(os.path.join(folder_path, file), 'r') as f:
        json_data = json.load(f)

    # Parse file name to extract dataset and model
    dataset, model = file.replace(".json", "").split("_values", 1)[1].split("_", 1)
    
    # Extract data from JSON
    record = {"dataset": dataset, "model": model}
    for key, value in json_data.items():
        if "max" in key:
            prefix, pred = key.split("_", 1)
            if prefix.startswith('dice'):
                record.update({f"{prefix}_{pred}": value})
    data.append(record)

# Create DataFrame
df = pd.DataFrame(data)

# Pivot DataFrame to create the desired table format
table = df.melt(id_vars=["dataset", "model"], var_name="metric_prediction", value_name="value")
table[['metric', 'prediction']] = table['metric_prediction'].str.rsplit("_", n=1, expand=True)
table = table.pivot_table(index=["dataset", "model", "prediction"], columns="metric", values="value").reset_index()
table.columns.name = None

# Reorder columns and remove "dice_" from metric names
columns = ['dataset', 'model', 'prediction', 'dice_cp', 'dice_rc', 'dice_rs3', 'dice_rs5', 'dice_bb', 'dice_bbs_05', 'dice_bbs_1', 'dice_bbs_2']
table = table[columns]
table.columns = [col.replace('dice_', '') for col in table.columns]

# Sort the DataFrame based on model and dataset
table['model'] = pd.Categorical(table['model'], categories=['vit-h', 'vit-l', 'vit-b'], ordered=True)
table.sort_values(['dataset', 'model', 'prediction'], inplace=True)
table.reset_index(drop=True, inplace=True)

print(table)

#save table
table.to_csv('/mnt/B-SSD/maltamed/SAM-zero-shot/code/max_dice.csv', index=False)


      dataset  model prediction        cp        rc       rs3       rs5   
0         CVC  vit-h        max  0.838099  0.853699  0.883536  0.898475  \
1         CVC  vit-l        max  0.814660  0.822783  0.848362  0.847285   
2         CVC  vit-b        max  0.738557  0.749223  0.782993  0.784425   
3   CXRkaggle  vit-h        max  0.921705  0.902340  0.927991  0.936385   
4   CXRkaggle  vit-l        max  0.929043  0.916694  0.931766  0.929629   
5   CXRkaggle  vit-b        max  0.914697  0.892543  0.929970  0.934934   
6         HAM  vit-h        max  0.782041  0.764185  0.811927  0.824413   
7         HAM  vit-l        max  0.783963  0.771816  0.809382  0.818572   
8         HAM  vit-b        max  0.745275  0.706321  0.785444  0.795828   
9        ISIC  vit-h        max  0.787564  0.767858  0.820197  0.835344   
10       ISIC  vit-l        max  0.782550  0.768335  0.811393  0.818296   
11       ISIC  vit-b        max  0.763711  0.732992  0.804399  0.814700   
12   hipfemur  vit-h     